In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import requests
from datetime import datetime, timedelta
import random
import pandas as pd
import base64
import time
import os

In [2]:
from datetime import datetime, timedelta

def format_tanggal_besok():
    # Tanggal hari ini
    today = datetime.now()
    
    # Tanggal besok
    tomorrow = today + timedelta(days=0)
    
    # Format tanggal besok dalam bahasa Inggris
    formatted_tomorrow = tomorrow.strftime("%d %B %Y")  # "07 December 2024"
    
    # Ubah bulan ke bahasa Indonesia
    bulan_inggris_ke_indonesia = {
        "January": "Januari",
        "February": "Februari",
        "March": "Maret",
        "April": "April",
        "May": "Mei",
        "June": "Juni",
        "July": "Juli",
        "August": "Agustus",
        "September": "September",
        "October": "Oktober",
        "November": "November",
        "December": "Desember"
    }
    
    for bulan_inggris, bulan_indonesia in bulan_inggris_ke_indonesia.items():
        formatted_tomorrow = formatted_tomorrow.replace(bulan_inggris, bulan_indonesia)
    
    return formatted_tomorrow

# Contoh penggunaan
# print(format_tanggal_besok())


In [ ]:

# Setup Chrome options
options = Options()
options.add_argument("--start-maximized")
options.add_argument("--disable-notifications")

# Path ke driver Chrome
service = webdriver.ChromeService()
# service = Service("C:\Program Files\Google\Chrome\Application\chrome.exe")  # Ganti dengan path driver

# Inisialisasi WebDriver
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://elearning.budiluhur.ac.id")

driver.implicitly_wait(10)

# Cari tombol berdasarkan class dan klik
button = driver.find_element(By.CLASS_NAME, "login-open")
button.click()

# WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, "username")))

driver.find_element(By.ID, "login-username").send_keys("2211510629")
driver.find_element(By.NAME, "password").send_keys("P29082003")

form = driver.find_element(By.TAG_NAME, "form")
form.submit()

target_url = "https://elearning.budiluhur.ac.id/mod/attendance/index.php?id=27186"
driver.get(target_url)


# driver.implicitly_wait(10)

# Temukan tabel berdasarkan ID
table = driver.find_element(By.CLASS_NAME, "generaltable")
print(table)

# Ambil semua baris dari tabel
rows = table.find_elements(By.TAG_NAME, "tr")

# Ekstrak data dari tabel
# data = []
# for row in rows:
#     cells = row.find_elements(By.TAG_NAME, "td")  # Ambil kolom
#     if not cells:  # Jika tidak ada <td>, ambil header
#         cells = row.find_elements(By.TAG_NAME, "th")
#     data.append([cell.text for cell in cells])


data = []
for row in rows:
    cells = row.find_elements(By.TAG_NAME, "td")  # Ambil kolom
    if not cells:  # Jika tidak ada <td>, ambil header
        cells = row.find_elements(By.TAG_NAME, "th")
    
    # List untuk menyimpan data baris
    row_data = []
    
    for index, cell in enumerate(cells):
        if index == 1:  # Jika kolom "Name" (indeks 1) berisi <a>
            link = None
            # Cek apakah ada elemen <a> dalam cell
            link_element = cell.find_elements(By.TAG_NAME, "a")
            if link_element:
                link = link_element[0].get_attribute("href")  # Ambil href dari <a>
            row_data.append(link if link else cell.text)  # Jika ada link, masukkan link, jika tidak masukkan teks
        else:
            row_data.append(cell.text)  # Untuk kolom lainnya, ambil teks saja
    
    data.append(row_data)

# Buat DataFrame dari data yang diambil
# print(data)
# Ambil tanggal besok dari fungsi
tanggal_besok = format_tanggal_besok()

# Filter data berdasarkan tanggal besok
filtered_data = [row for row in data if tanggal_besok in row[0]]

# Print hasil filter
print(filtered_data)
driver.get(filtered_data[0][1])

# Buka halaman berdasarkan hasil filter
if filtered_data:
    driver.get(filtered_data[0][1])
    print("Berpindah ke halaman:", filtered_data[0][1])
else:
    print("Tidak ada data yang cocok untuk tanggal:", tanggal_besok)




# # Cari tabel berdasarkan class name
# table = driver.find_element(By.CLASS_NAME, "generaltable")

# # Ambil semua baris dari tabel (termasuk header)
# rows = table.find_elements(By.TAG_NAME, "tr")

# # Loop melalui baris dan ekstrak data
# for row in rows:
#     # Ambil semua kolom dalam baris (td atau th)
#     cells = row.find_elements(By.TAG_NAME, "td")
#     # Jika tidak ada kolom <td>, itu adalah header, jadi ambil <th>
#     if not cells:
#         cells = row.find_elements(By.TAG_NAME, "th")
    
#     # Print data kolom
#     for cell in cells:
#         print(cell.text)
# -------------------------------------
# Cari tabel berdasarkan class name
# Temukan tabel berdasarkan kelas
table = driver.find_element(By.CLASS_NAME, "generaltable")

# Ambil semua baris dari tabel
rows = table.find_elements(By.TAG_NAME, "tr")

# Loop untuk setiap baris
for row in rows:
    # Ambil semua kolom dalam baris (td atau th)
    cells = row.find_elements(By.TAG_NAME, "td")
    
    # Cek jika kolom Status berisi link
    if len(cells) > 2:  # Memastikan ada kolom Status
        status_cell = cells[2]  # Kolom ke-3 adalah Status
        try:
            link_element = status_cell.find_element(By.TAG_NAME, "a")  # Temukan <a> dalam Status
            link = link_element.get_attribute("href")  # Ambil href dari link
            print("Link:", link)
            
            # Pindah ke halaman dari link yang ditemukan
            driver.get(link)
            break  # Keluar setelah berpindah halaman
        except:
            print("Tidak ada link di kolom Status")
            driver.quit()  # Jika tidak ada link, tutup browser
            break  # Keluar dari loop

# Tunggu sampai elemen tersedia
driver.implicitly_wait(10)

# Temukan radio button berdasarkan label yang mengandung "Hadir" (atau label lain yang relevan)
radio_button = driver.find_element(By.XPATH, "//span[text()='Hadir']/preceding-sibling::input[@type='radio']")
radio_button.click()  # Pilih radio button "Hadir"

# Temukan tombol submit berdasarkan value="Save changes"
submit_button = driver.find_element(By.XPATH, "//input[@value='Save changes']")
submit_button.click()  # Klik tombol "Save changes"


# df = pd.DataFrame(data[1:], columns=data[0])  # Baris pertama sebagai header
# # print(df)
# # print(df['Topic'])

# # Tanggal hari ini
# # today = datetime.now().strftime("%d %B %Y")  # Format: "12 Desember 2024"
# # tomorrow = today + timedelta(days=1)
# today = format_tanggal_besok()

# # Ekstrak tanggal dari kolom topic
# df['tanggal'] = df['Topic'].str.extract(r'\|\s*(.*)$')

# # Filter DataFrame sesuai tanggal hari initoday
# filtered_df = df[df['tanggal'] == today]

# print(today)

# # Hasil
# # Misalkan filtered_df berisi DataFrame yang sudah difilter, dengan kolom 'Name' yang berisi URL
# for index, row in filtered_df.iterrows():
#     link = row['Name']  # Ambil URL dari kolom 'Name'
#     print(link)  # Opsional: menampilkan URL
#     driver.get(link)  # Arahkan browser ke URL yang ada di kolom 'Name'


# Cetak DataFrame
# print(df)

# Tutup driver setelah selesai
driver.quit()
# df

<selenium.webdriver.remote.webelement.WebElement (session="191453ffc921f70a68cbaa0daa761542", element="f.0140BDEB00257404CADDE84F485A2237.d.97D429902EAE5955AFC36F2C11C56E90.e.81")>
[['Pertemuan 10 | 07 Desember 2024', 'https://elearning.budiluhur.ac.id/mod/attendance/view.php?id=1264524']]
Berpindah ke halaman: https://elearning.budiluhur.ac.id/mod/attendance/view.php?id=1264524
Tidak ada link di kolom Status
